# dependencies
* install anaconda is recommended

```
cassandra-driver          3.11.0                   py35_1    conda-forge
pandas                    0.19.1              np111py35_0
scikit-learn              0.18.1              np111py35_0
scipy                     0.18.1              np111py35_0
matplotlib                2.0.0               np111py35_0
```

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib
import time
import numpy as np
%matplotlib inline

import pandas as pd
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100

# import training as training

In [2]:
#Model File Directory
WORK_DIR = '/opt/docker/workspace/ml/'

#Label Keys
LABEL = "success"

MODEL_TYPE = "ML-BR"

start_date = '2018-01-01'
end_date = '2018-03-31'
site_ids = ['adbecnn', 'adbehap']  #['avast', 'kasperus', 'mcafeeus', 'mfeap', 'mfeeu']
desc = str('%s_%s_%s' % (start_date, end_date, "_".join(site_ids)))

In [3]:
# from cassandra
from cassandra.cluster import Cluster
cassandra_endpoint = '10.224.12.32'
cluster = Cluster([cassandra_endpoint])
session = cluster.connect('subs')


In [4]:
import datetime
from datetime import timedelta
import time

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import preprocessing
import calendar, datetime
import time
from sklearn.preprocessing import LabelEncoder

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))

class PreProcessing(BaseEstimator, TransformerMixin):
    """Custom Pre-Processing estimator for SRS
    """

    def __init__(self):
        pass

    def transform(self, df):
        """Regular transform() that is a help for training, validation & testing datasets
           (NOTE: The operations performed here are the ones that we did prior to this cell)
        """
        # Consolidated feature processing
        
        df['week_of_month'] = df['transaction_date_in_string'].apply(week_of_month)
        df['day_of_week'] =  df['transaction_date_in_string'].apply(to_weekday)
        # one hot for categorical feature ###
#         df_encoded = pd.get_dummies(df[FEATURES].astype(str), prefix=FEATURES)
#         df_encoded = pd.DataFrame(df_encoded, columns=self.features_list).fillna(0)

        df_encoded  = pd.DataFrame(columns=self.features_list)
        for k, v in self.encoders.items():
            if df[k].dtype == 'float64':
                df[k] = df[k].fillna(-1).astype(int)
            
            df_encoded[k] = v.transform(df[k].astype(str).str.lower().str.replace(' ',''))
        
        #Num processing
        df_num = df[self.FEATURES_NUM].astype(float)
        df_num = self.scaler.transform(df_num)

        df_encoded[self.FEATURES_NUM] = df_num

        return df_encoded.as_matrix()

    def fit(self, df, y=None, features_dict={}, **fit_params):
        
        print('features_dict: ', features_dict)
        self.FEATURES_CAT = features_dict['FEATURES_CAT']
        self.FEATURES_NUM = features_dict['FEATURES_NUM']
        self.FEATURES_ENCODED = features_dict['FEATURES_ENCODED']
        self.FEATURES = self.FEATURES_CAT + self.FEATURES_ENCODED
        
        df['week_of_month'] = df['transaction_date_in_string'].apply(week_of_month)
        df['day_of_week'] =  df['transaction_date_in_string'].apply(to_weekday)

        
        # one hot for categorical feature ###
#         self.features_list = list(pd.get_dummies(df[FEATURES].astype(str), prefix=FEATURES).columns.values) + FEATURES_NUM

        self.features_list = self.FEATURES + self.FEATURES_NUM
        print("self.features_list: ", self.features_list)
        feature_encoders = {}
        for f in self.FEATURES:
            if df[f].dtype == 'float64':
                df[f] = df[f].fillna(-1).astype(int)
                
            encoder = SafeLabelEncoder().fit(df[f].astype(str).str.lower().str.replace(' ',''))
            feature_encoders[f] = encoder
            
        self.encoders = feature_encoders  

        #Fit a scaler
        df_num = df[self.FEATURES_NUM].astype(float)
        self.scaler = preprocessing.StandardScaler().fit(df_num)
        return self


In [5]:
import itertools

def get_site_date_cartesian_products(site_ids, dates):
    return itertools.product(*[site_ids, dates])

def query_raw_data(start, end, site_ids):
    """ Queries cpg raw data based on the given start, end date and list of site_ids """
    
    _result = pd.DataFrame()
    query = "SELECT * FROM cpg_transaction where received_date = '%s' and site_id = '%s'"
    dates = daterange(start, end)
    for site_id, date in get_site_date_cartesian_products(site_ids, dates):
        query_by_date = str(query % (date.replace('-',''), site_id))
        print(query_by_date)
        _result = pd.concat([_result, pd.DataFrame(list(session.execute(query_by_date)))] )
    
    return _result

""" Load the raw data into df_cs_original """
df_cs_original = query_raw_data(start_date, end_date, site_ids)
csv_name = str('df_cs_original_%s_%s_%s.csv' % (start_date, end_date, "_".join(site_ids)))
df_cs_original.to_csv(WORK_DIR + csv_name)

df_cs_original.shape

SELECT * FROM cpg_transaction where received_date = '20180101' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where received_date = '20180102' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where received_date = '20180103' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where received_date = '20180104' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where received_date = '20180105' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where received_date = '20180106' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where received_date = '20180107' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where received_date = '20180108' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where received_date = '20180109' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where received_date = '20180110' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where received_date = '20180111' and site_id = 'adbecnn'
SELECT * FROM cpg_transaction where receive

SELECT * FROM cpg_transaction where received_date = '20180106' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180107' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180108' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180109' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180110' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180111' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180112' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180113' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180114' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180115' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180116' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where receive

(778783, 47)

In [6]:
csv_name = str('df_cs_original_%s_%s_%s.csv' % (start_date, end_date, "_".join(site_ids)))
df_cs_original.to_csv(WORK_DIR + csv_name)

In [17]:
df_cs_original['site_id'].value_counts()

adbehap    775326
adbecnn      3457
Name: site_id, dtype: int64

In [7]:
# Get only data that failed at first attempt

df_nl = df_cs_original
# Add 'success' as column
df_nl['success'] = df_nl['new_status'].map({'Completed':1,'Declined':0, 'Failed':0, 'Reversed':1})
# Remove renewal data that success at first attempt
df_nl = df_nl[~((df_nl['renew_att_num'] == '1') & (df_nl['success'] == 1))]
df_nl.shape

(263814, 48)

In [8]:
import datetime

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   

def merge_by_sub(group):

    first = group[ group['renew_att_num'] == '1' ]
    
    if first.empty:
        return pd.DataFrame({})
    
    last = group[ group['renew_att_num'] == group['renew_att_num'].max() ]
        
    first_attempt_date = first['transaction_date_in_string'].iloc[0]
    last_attempt_date = last['transaction_date_in_string'].iloc[0]
    
    last['first_attempt_date'] = first_attempt_date
    last['first_day_of_month'] = first['day_of_month'].iloc[0]
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['first_transaction_hour'] = first['transaction_hour'].iloc[0]
    last['first_response_code'] = first['response_code'].iloc[0]
    
    first_response_msg = first['response_message'].iloc[0]
    last['first_response_message'] = first_response_msg  
    last['first_response_group'] =   first['response_message'].apply(group_response_msg).iloc[0]
    
    last['first_decline_type'] = decline_type(first_response_msg)
    last['days_between'] =  days_between(first_attempt_date, last_attempt_date)
        
    return last

def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

In [9]:
df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

In [10]:
#Group by subscription_id for failed data at first attempt
time_start = time.time()
df_nl = df_nl[~df_nl['renew_att_num'].isna()]
df_subs_merged = pd.DataFrame()
for site_id in site_ids:
    df_subs = df_nl[df_nl['site_id'] == site_id].groupby(['subscription_id', 'subsegment_id'], sort=False)
#     df_subs_merged = df_subs.apply(merge_by_sub)
    df_subs_merged = pd.concat([df_subs_merged, df_subs.apply(merge_by_sub)], axis=0)
    
print("# merge time:", time.time() - time_start)
df_subs_merged.shape

# merge time: 550.4875392913818


(76643, 56)

In [11]:
#Only take subscription that has at least two transactions that occured on different day
retry_success = df_subs_merged[df_subs_merged['days_between'] > 0]

# Drop duplicate columns
retry_success = retry_success.drop(columns=['subscription_id', 'subsegment_id'])

#Write merged subs to csv
csv_name = str('subs_retry_%s_%s_%s.csv' % (start_date, end_date, "_".join(site_ids)))
retry_success.to_csv(WORK_DIR + csv_name)
print("Saved into " + WORK_DIR + csv_name)

Saved into /opt/docker/workspace/ml/subs_retry_2018-01-01_2018-03-31_adbecnn_adbehap.csv


In [12]:
### resampling, balancing classes ###
def balancing_class(df_ml):
    print(df_ml[LABEL].value_counts(normalize=True))
    df_cls_0 = df_ml[df_ml[LABEL] == 0]
    df_cls_1 = df_ml[df_ml[LABEL] == 1]

    #over sampling
#     df_cls_0_over = df_cls_0.sample(len(df_cls_1), replace=True)
#     df_ml_bl = pd.concat([df_cls_0_over, df_cls_1], axis=0)
    df_cls_1_over = df_cls_1.sample(len(df_cls_0), replace=True)
    df_ml_bl = pd.concat([df_cls_1_over, df_cls_0], axis=0)

#     #under sampling
#     df_cls_1_under = df_cls_1.sample(len(df_cls_0), replace=True)
#     df_ml_bl = pd.concat([df_cls_1_under, df_cls_0], axis=0)

    print(df_ml_bl[LABEL].value_counts(normalize=True))
    print(df_ml_bl.shape)
    return df_ml_bl

In [14]:
retry_success_balanced_all = balancing_class(retry_success)
csv_name = str('subs_retry_balanced__%s_%s_%s.csv' % (start_date, end_date, "_".join(site_ids)))
retry_success_balanced_all.to_csv(WORK_DIR + csv_name)
print("Saved into " + WORK_DIR + csv_name)
retry_success_balanced_all.shape

0.0    0.687066
1.0    0.312934
Name: success, dtype: float64
0.0    0.5
1.0    0.5
Name: success, dtype: float64
(92376, 54)
Saved into /opt/docker/workspace/ml/subs_retry_balanced__2018-01-01_2018-03-31_adbecnn_adbehap.csv


(92376, 54)

In [16]:
original_size = len(retry_success)
balanced_size = len(retry_success_balanced_all)

92376

In [17]:
#import for training
import numpy as np
from sklearn import cross_validation
from sklearn import metrics
from sklearn.model_selection import cross_val_score

from sklearn import linear_model
from sklearn import tree
from sklearn import cross_validation
from sklearn import ensemble
from sklearn import linear_model
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
# from spark_sklearn import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier

/drhdev/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import column_or_1d
from sklearn.preprocessing.label import _check_numpy_unicode_bug

def _get_unseen():
    """Basically just a static method
    instead of a class attribute to avoid
    someone accidentally changing it."""
    return 99999

class SafeLabelEncoder(LabelEncoder):
    """An extension of LabelEncoder that will
    not throw an exception for unseen data, but will
    instead return a default value of 99999
    Attributes
    ----------
    classes_ : the classes that are encoded
    """

    def transform(self, y):
        """Perform encoding if already fit.
        Parameters
        ----------
        y : array_like, shape=(n_samples,)
            The array to encode
        Returns
        -------
        e : array_like, shape=(n_samples,)
            The encoded array
        """
        check_is_fitted(self, 'classes_')
        y = column_or_1d(y, warn=True)

        classes = np.unique(y)
        _check_numpy_unicode_bug(classes)

        # Check not too many:
        unseen = _get_unseen()
        if len(classes) >= unseen:
            raise ValueError('Too many factor levels in feature. Max is %i' % unseen)

        e = np.array([
                         np.searchsorted(self.classes_, x) if x in self.classes_ else unseen
                         for x in y
                         ])

        return e

In [19]:
from sklearn import cross_validation

def print_accuracy_report(classifier, X, y, num_validations=5):
    accuracy = cross_validation.cross_val_score(classifier, 
            X, y, scoring='accuracy', cv=num_validations)
    print("CV Accuracy: " + str(round(100*accuracy.mean(), 2)) + "%")

    f1 = cross_validation.cross_val_score(classifier, 
            X, y, scoring='f1_weighted', cv=num_validations)
    print("CV F1: " + str(round(100*f1.mean(), 2)) + "%")

    precision = cross_validation.cross_val_score(classifier, 
            X, y, scoring='precision_weighted', cv=num_validations)
    print("CV Precision: " + str(round(100*precision.mean(), 2)) + "%")

    recall = cross_validation.cross_val_score(classifier, 
            X, y, scoring='recall_weighted', cv=num_validations)
    print("CV Recall: " + str(round(100*recall.mean(), 2)) + "%")

In [20]:
FEATURES_CAT = ['card_brand', 'funding_source', 'card_category', 'card_class', 
                 'issuer_country', 'first_response_code', 'day_of_month',
               'first_decline_type',  'mid_entity_code', 'payment_service_id', 'payment_method_id', 'bin', 'first_day_of_month']

FEATURES_NUM = ['payment_amount_usd']
FEATURES_ENCODED = [ 'week_of_month', 'day_of_week']
FEATURES = FEATURES_CAT + FEATURES_ENCODED


# from sklearn.model_selection import GridSearchCV
from spark_sklearn import GridSearchCV

# from training import PreProcessing
from sklearn.pipeline import make_pipeline


FIELDS = FEATURES_CAT + FEATURES_NUM + ['transaction_date_in_string']

features_dict = {'FEATURES_CAT': FEATURES_CAT, 'FEATURES_NUM':FEATURES_NUM, 'FEATURES_ENCODED':FEATURES_ENCODED}
features_dict_key = 'preprocessing__features_dict'
def display_feature_importance(pipe, model_name, df_features):
    classifier = pipe.named_steps[model_name]
    feature_importance = classifier.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    print("feature_importance column ",df_features.columns[sorted_idx])
    print("feature_importance val ",feature_importance[sorted_idx])
    pos = np.arange(sorted_idx.shape[0]) + .5
    pvals = feature_importance[sorted_idx]
    pcols = df_features.columns[sorted_idx]
    plt.figure(figsize=(8,12))
    plt.barh(pos, pvals, align='center')
    plt.yticks(pos, pcols)
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')

def build_and_train(df, clf, param_grid, model_name, model_file = ''):
    model_prefix = model_name + '__'
    time_start = time.time()
    df_X = df[FIELDS]
 
    x_train, x_test, y_train, y_test = cross_validation.train_test_split(df_X, df[LABEL], \
                                                        test_size=0.25, random_state=42)

    pipe = make_pipeline(PreProcessing(), clf())

    score = 'accuracy' #  ['accuracy','precision_macro', 'recall_macro', 'f1_macro']

    print("# Tuning hyper-parameters for %s" % score)
        
    pipe_param_grid = {model_prefix + k: v for k, v in param_grid.items()}
    print("pipe_param_grid ", pipe_param_grid)
    clf_gs = GridSearchCV(sc, pipe, pipe_param_grid, cv=3, scoring=score, n_jobs=-1, fit_params={features_dict_key: features_dict})
#     clf_gs = GridSearchCV(pipe, pipe_param_grid, cv=5, scoring=score, n_jobs=-1, fit_params={features_dict_key: features_dict})
    clf_gs.fit(x_train, y_train)

    print("# Best parameters set found on development set:")
    print(clf_gs.best_params_)
    print("# Grid scores on development set:")
    means = clf_gs.cv_results_['mean_test_score']
    stds = clf_gs.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf_gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

    print("x_train", x_train.shape)
    print("x_test", x_test.shape)

    best_parameters = clf_gs.best_params_
    best_parameters = {k.replace(model_prefix,''): v for k, v in best_parameters.items()}
    print("best_parameters ", best_parameters)
    pipe = make_pipeline(PreProcessing(), clf(**best_parameters))
    
    if model_name == 'xgbclassifier':
        print("training xgb ....... ")
        if model_file == '':
            pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict, xgbclassifier__eval_metric='error')
        else:
            print("Using model_file to train: ", model_file)
            pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict, xgbclassifier__xgb_model=model_file, xgbclassifier__eval_metric='error')
    else:        
        pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict)
        
    preprocess = pipe.named_steps['preprocessing']

        
    y_pred_train = pipe.predict(x_train).round()
    y_pred_test = pipe.predict(x_test).round()
    
    print("# training time:", time.time() - time_start)

    clf_d = DummyClassifier(strategy='most_frequent')
    dummy_pipe = make_pipeline(PreProcessing(), clf_d)
    dummy_pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict)
    y_pred_test_dummy = dummy_pipe.predict(x_test).round()
    print("accuracy_dummy:", metrics.accuracy_score(y_test, y_pred_test_dummy))
    print("training accuracy:", metrics.accuracy_score(y_train, y_pred_train))
    print("test accuracy:", metrics.accuracy_score(y_test, y_pred_test))
     
#     print_accuracy_report(pipe, x_train, y_train, num_validations=3)
    conf_mx = metrics.confusion_matrix(y_test, y_pred_test)
    print("# confusion_matrix -  test:\n", conf_mx)
    

    return pipe


In [21]:
# save model
from sklearn.externals import joblib
import os
from os import path

MODEL_DIR = WORK_DIR + "models"


def write_model(model, model_name, idx=None): 
    build_id = "" if idx is None else "_" + str(idx)
    file_name = path.join(MODEL_DIR, '%s%s.pkl' % (model_name, build_id))
    if not os.path.exists(os.path.dirname(file_name)):
        os.makedirs(os.path.dirname(file_name))
    file = joblib.dump(model, file_name)
    return (file, file_name)

In [22]:
from cassandra.cluster import Cluster
cassandra_endpoint = '10.224.12.32'
cluster = Cluster([cassandra_endpoint])
session = cluster.connect('subs')


def insert_model_info(model_id, version, file_name, desc, model_type=MODEL_TYPE, algorithm='XGBClassifier', hyper_parameter=None, eval_metrics=None):
#     file_name = model_id + '.' + str(version) + ".pkl"
    
    session.execute(
    """
    INSERT INTO ml_model_storage (model_type, model_id, version, features_cat, features_encoded, features_num, repo_path, description, creation_date, modification_date, algorithm, hyper_parameter, eval_metrics)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
    (model_type, model_id, version, str(FEATURES_CAT), str(FEATURES_ENCODED), str(FEATURES_NUM), file_name, desc, datetime.datetime.utcnow(), datetime.datetime.utcnow(), algorithm, hyper_parameter, eval_metrics)
        
    )
    print("Model %s version %d is inserted into model repo" % (model_id, version))      


In [23]:
def get_latest_version(model_id, model_type=MODEL_TYPE):
    """Get latest version of the given model_id"""
    latest_version_query = "select version from subs.ml_model_storage  where model_type = '%s' and model_id = '%s' limit 1" % (model_type, model_id)
    
    query_result = session.execute(latest_version_query).one()
    if query_result is None:
        latest_version = 1
    else:
        latest_version = query_result.version
    
    return latest_version



In [24]:
REPOSITORY_URL = 'http://nexus.digitalriverws.net/nexus'
REPO_USER = 'deployment'
REPO_PWD = 'deployment123'
REPO_ID = 'foundationreleases'
REPO_GROUP = 'com.digitalriver.srs-ml'

import repositorytools

def upload_artifact(file_path):
    """Upload artifact to Nexus Repo"""
    artifact = repositorytools.LocalArtifact(local_path=file_path, group=REPO_GROUP)

    client = repositorytools.repository_client_factory(repository_url=REPOSITORY_URL, user=REPO_USER, password=REPO_PWD)
    remote_artifacts = client.upload_artifacts(local_artifacts=[artifact], repo_id=REPO_ID, use_direct_put=True)
    print(remote_artifacts)
    return str(remote_artifacts[0]) if remote_artifacts else ''

In [25]:
""" Train the model"""

#XGBoost Classifier
from xgboost import XGBClassifier
# import xgboost as xgb
classifier = XGBClassifier

tuned_parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.2], #so called `eta` value
              'max_depth': [10],
              'min_child_weight': [11],
              'silent': [0],
              'subsample': [0.5],
              'colsample_bytree': [0.7],
              'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}

model_file = ''
# for idx, df_2018_f in enumerate(df_2018_filtered):
#     print("Iteration: ", idx)
model_id = 'ML-BR-1'
version = get_latest_version(model_id) + 1
model_name = model_id + '.' + str(version)
xgb_clf = build_and_train(retry_success_balanced_all, classifier, tuned_parameters, 'xgbclassifier', model_file)
model_file, model_file_name = write_model(xgb_clf, model_name)
print("model_file is generated: ", model_file)

# insert_model_info(model_id, version, model_file_name, desc="For testing")

# Tuning hyper-parameters for accuracy
pipe_param_grid  {'xgbclassifier__objective': ['binary:logistic'], 'xgbclassifier__learning_rate': [0.2], 'xgbclassifier__max_depth': [10], 'xgbclassifier__min_child_weight': [11], 'xgbclassifier__silent': [0], 'xgbclassifier__subsample': [0.5], 'xgbclassifier__colsample_bytree': [0.7], 'xgbclassifier__n_estimators': [500, 1000], 'xgbclassifier__missing': [-999], 'xgbclassifier__seed': [1337]}
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'issuer_country', 'first_response_code', 'day_of_month', 'first_decline_type', 'mid_entity_code', 'payment_service_id', 'payment_method_id', 'bin', 'first_day_of_month'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week']}


/drhdev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/drhdev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'issuer_country', 'first_response_code', 'day_of_month', 'first_decline_type', 'mid_entity_code', 'payment_service_id', 'payment_method_id', 'bin', 'first_day_of_month', 'week_of_month', 'day_of_week', 'payment_amount_usd']


/drhdev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/drhdev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/drhdev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

# Best parameters set found on development set:
{'xgbclassifier__colsample_bytree': 0.7, 'xgbclassifier__learning_rate': 0.2, 'xgbclassifier__max_depth': 10, 'xgbclassifier__min_child_weight': 11, 'xgbclassifier__missing': -999, 'xgbclassifier__n_estimators': 1000, 'xgbclassifier__objective': 'binary:logistic', 'xgbclassifier__seed': 1337, 'xgbclassifier__silent': 0, 'xgbclassifier__subsample': 0.5}
# Grid scores on development set:
0.901 (+/-0.003) for {'xgbclassifier__colsample_bytree': 0.7, 'xgbclassifier__learning_rate': 0.2, 'xgbclassifier__max_depth': 10, 'xgbclassifier__min_child_weight': 11, 'xgbclassifier__missing': -999, 'xgbclassifier__n_estimators': 500, 'xgbclassifier__objective': 'binary:logistic', 'xgbclassifier__seed': 1337, 'xgbclassifier__silent': 0, 'xgbclassifier__subsample': 0.5}
0.907 (+/-0.001) for {'xgbclassifier__colsample_bytree': 0.7, 'xgbclassifier__learning_rate': 0.2, 'xgbclassifier__max_depth': 10, 'xgbclassifier__min_child_weight': 11, 'xgbclassifier__mi

...

# training time: 573.465653181076
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'issuer_country', 'first_response_code', 'day_of_month', 'first_decline_type', 'mid_entity_code', 'payment_service_id', 'payment_method_id', 'bin', 'first_day_of_month'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week']}


/drhdev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/drhdev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'issuer_country', 'first_response_code', 'day_of_month', 'first_decline_type', 'mid_entity_code', 'payment_service_id', 'payment_method_id', 'bin', 'first_day_of_month', 'week_of_month', 'day_of_week', 'payment_amount_usd']


/drhdev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/drhdev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


accuracy_dummy: 0.49636269160820995
training accuracy: 0.9865477324557606
test accuracy: 0.9263878063566294
# confusion_matrix -  test:
 [[10356  1107]
 [  593 11038]]
model_file is generated:  ['/opt/docker/workspace/ml/models/ML-BR-1.22.pkl']


In [26]:
"""Upload model to Nexus repo and insert the model info into Cassandra table"""

repo_path = upload_artifact(model_file_name)
size_desc = str(", original size: %s, balanced_size: %s" % (original_size, balanced_size))
desc = desc + size_desc
insert_model_info(model_id, version, repo_path, desc=desc, eval_metrics="training accuracy: 0.9793473731754407, test accuracy: 0.9655285933757646", 
                  hyper_parameter="{'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 10, 'min_child_weight': 11, 'missing': -999, 'n_estimators': 1000, 'objective': 'binary:logistic', 'seed': 1337, 'silent': 0, 'subsample': 0.5}")

http://nexus.digitalriverws.net/nexus/content/repositories/foundationreleases/com/digitalriver/srs-ml/ML-BR/1.22/ML-BR-1.22.pkl
[com.digitalriver.srs-ml:ML-BR:1.22::pkl]
Model ML-BR-1 version 22 is inserted into model repo


The following files were uploaded to repository foundationreleases
